<a href="https://colab.research.google.com/github/YaoYinYing/PPRCODE_Guideline/blob/master/PPRCODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PPRCODE on Colab 
This is a experimentally reimplement work created by Yinying Yao.

Original webserver (MAY not work): http://yinlab.hzau.edu.cn/pprcode/

**If you find this tool useful to your research, please cite the following**:

- Junjie Yan, Yinying Yao, Sixing Hong, Yan Yang, Cuicui Shen, Qunxia Zhang, Delin Zhang, Tingting Zou, Ping Yin; Delineation of pentatricopeptide repeat codes for target RNA prediction, Nucleic Acids Research. PMID:30753696 DOI: 10.1093/nar/gkz075




In [ ]:
#@title Install packages
import os
import glob
import pathlib
import sys
import time
import traceback
import pickle
import re
from IPython.utils import io
import subprocess
import tqdm.notebook

try:
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False


PROSITE_URL='https://ftp.expasy.org/databases/prosite/'
PS_SCAN_URL='https://ftp.expasy.org/databases/prosite/ps_scan/ps_scan_linux_x86_elf.tar.gz'
PS_MOTIF='PS51375'
TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

TMP_DIR = "tmp"
JOBS_DIR='jobs'
RES_DIR='results'
RES_DIR_SCAN=f"{RES_DIR}/scan"
RES_DIR_PICKLE=f'{RES_DIR}/pickle'
RES_DIR_REPORT=f'{RES_DIR}/report'



# if not already installed
total = 10
with tqdm.notebook.tqdm(total=total, bar_format=TQDM_BAR_FORMAT) as pbar:
  for dir in [TMP_DIR,JOBS_DIR,RES_DIR,RES_DIR_SCAN,RES_DIR_PICKLE,RES_DIR_REPORT]:
    os.makedirs(dir, exist_ok=True)
  pbar.update(3)
  os.system(f"wget -qnc {PS_SCAN_URL} -P {TMP_DIR}")
  pbar.update(1)
  os.system(f'mkdir {TMP_DIR}/bin && cd {TMP_DIR}/bin && tar zxf ../ps_scan_linux_x86_elf.tar.gz')
  pbar.update(1)
  os.system(f'mkdir {TMP_DIR}/profiles')
  pbar.update(1)

  for p in ['profile.txt','prosite.dat','prorule.dat','evaluator.dat']:
    os.system(f"wget -qnc {PROSITE_URL}{p} -P {TMP_DIR}/profiles/")
  pbar.update(1)

  try:
    from Bio import SeqIO, Seq
  except:
    os.system(f"pip3 install biopython")
    from Bio import SeqIO,Seq
  pbar.update(1)

  try:
    import xlsxwriter as xw
  except:
    os.system(f"pip3 install xlsxwriter")
  import xlsxwriter as xw
  pbar.update(1)

  if f"{pathlib.Path(TMP_DIR).resolve()}/bin/ps_scan/" not in os.environ['PATH']:
    os.environ['PATH'] += f":{TMP_DIR}/bin/ps_scan"
  pbar.update(1)

# pprcode to rna base reflection
codeset = ["ND", "NN", "TD", "SD", "TN", "SN", "NS", "NT", "GN", "TS", "GD", "TT", "RD", "SS", "AD", "KD", "ST", "NE",
           "DS", "CD", "DD", "NG", "AN", "GS", "ES", "LD", "SG", "GT", "HD", "CN", "VD", "ID", "SE", "NC", "AS", "RS",
           "MD", "NK", "NL", "DT", "RN", "TG", "GG", "LN", "LS", "VN", "TE", "SP", "KN", "NR", "DN", "CS", "QD", "NV",
           "ED", "IN", "VS", "HN", "AT", "NH", "AG", "NI", ]

# updated at 2018/12/20 for pulication, experimentally validated.
code2rna_multi = {"ND": "U>C>G", "NN": "C>U", "TD": "G>A>U", "SD": "G>>C", "TN": "A>G", "SN": "A", "NS": "C>U>A",
                  "NT": "C>U", "GN": "A>C", "TS": "A>G", "GD": "G", "TT": "A>G", "RD": "-", "SS": "A", "AD": "G",
                  "KD": "-", "ST": "A", "NE": "U>C", "DS": "-", "CD": "G", "DD": "-", "NG": "C>U>A", "AN": "A>>U",
                  "GS": "-", "ES": "-", "LD": "-", "SG": "A>G", "GT": "C", "HD": "-", "CN": "A>>G", "VD": "G",
                  "ID": "-", "SE": "-", "NC": "U>C>>A", "AS": "-", "RS": "-", "MD": "-", "NK": "C>U", "NL": "C",
                  "DT": "-", "RN": "-", "TG": "A>>G", "GG": "-", "LN": "-", "LS": "-", "VN": "A", "TE": "G", "SP": "-",
                  "KN": "C>U", "NR": "-", "DN": "C", "CS": "A", "QD": "-", "NV": "C>U", "ED": "-", "IN": "-",
                  "VS": "-", "HN": "-", "AT": "-", "NH": "U>C", "AG": "-", "NI": "C", }

def ps_scan(f,s):
  
    if debug: print(s.id)
    sequence_=s
    #print(sequence_)
    ps_out_file=f"{RES_DIR_SCAN}/{s.id}_PPR_PSSCAN_pff.log"
    cmd=f"perl {pathlib.Path(TMP_DIR).resolve()}/bin/ps_scan/ps_scan.pl -e {s.id} -p {PS_MOTIF} -o pff -d {pathlib.Path(TMP_DIR).resolve()}/profiles/prosite.dat {f} | sort -n -k 2 > {ps_out_file}"
    os.system(cmd)
    ps_out_file_scan=f"{RES_DIR_SCAN}/{s.id}_PPR_PSSCAN_scan.log"
    cmd=f"perl {pathlib.Path(TMP_DIR).resolve()}/bin/ps_scan/ps_scan.pl -e {s.id} -p {PS_MOTIF} -o scan -d {pathlib.Path(TMP_DIR).resolve()}/profiles/prosite.dat {f} | sort -n -k 2 > {ps_out_file_scan}"
    os.system(cmd)
    try:
        
        result_response_pff=open(ps_out_file,'r').read()
        matchpos = re.findall(r'(\d+)\t(\d+)\t%s' % PS_MOTIF, result_response_pff)
        #print(matchpos)
        refinedpos = [[], []]

        # print(matchpos)
        match_motif = []

        # get original match motifs
        for pos in matchpos:
            mypos=sequence_[int(pos[0])-1:int(pos[1])]
            mypos.id=f'{s.id}_{int(pos[0])-1}_{pos[1]}'
            #print(mypos)
            match_motif.append(mypos)

        #print(match_motif)
        # find the motif sequence deletion by ProSite and refine the positions
        
        result_response_scan=open(ps_out_file_scan,'r').read()
        for origin_motif, origin_pos in zip([str(x.seq) for x in match_motif], matchpos):
            try:
                if debug: print(origin_motif)
                if fix_gap: 
                  motif_delete_find = re.findall(r'' + origin_motif + '(-+)', result_response_scan)[0]
                else:
                  motif_delete_find=''
                if debug: print("orginal Motif= %s\t motif deletion=%s\t deletion length=%s" % (origin_motif, motif_delete_find,len(motif_delete_find)))
                refinedpos[0].append(int(origin_pos[0]))

                # V1.3.1 solve the motif boundary overlap
                if (len(refinedpos[1]) > 0 and refinedpos[1][-1] > refinedpos[0][-1]):
                    if debug: print("==>refine pos 0 = %s\t refine pos 1 = %s" % (refinedpos[0][-1], refinedpos[1][-1]))
                    refinedpos[1][-1] = refinedpos[0][-1]
                    if debug: print("-->refine pos 0 = %s\t refine pos 1 = %s" % (refinedpos[0][-1], refinedpos[1][-1]))


                refinedpos[1].append(int(origin_pos[1]) + 1 + len(motif_delete_find))

            except Exception as e:
                #traceback.print_exc()
                #if debug: print("orginal Motif= %s\t motif deletion=%s" % (origin_motif, "deletion not found."))
                refinedpos[0].append(int(origin_pos[0]))
                refinedpos[1].append(int(origin_pos[1]) + 1)



        # the second round of refinement of position: the conserved "P" and the 36 aa length motif  # the conserved "P" label is based on weblogo results
        # Eureka!
        refindpos_end_2nd = []
        try:
            pos_index = 0
            for refinedpos_start_slide, refinedpos_end_slide in zip(refinedpos[0], refinedpos[1]):
                if debug: print("refined pos[%s]: %s -> %s " % (
                    pos_index, refinedpos_start_slide, refinedpos_end_slide))
                if debug: print(sequence[refinedpos_start_slide:refinedpos_end_slide])
                if debug: print("Scanning the %s/%s motif..." % (pos_index + 1, refinedpos[0].__len__()))
                if pos_index < (refinedpos[0].__len__() - 1):
                    try:
                      if (refinedpos[0][pos_index + 1] - refinedpos[1][
                          pos_index] == 1 and
                              sequence[refinedpos_start_slide:refinedpos_end_slide][-1] == "P"):
                          if debug: print("Find P!")
                          if debug: print(len(sequence[refinedpos_start_slide:refinedpos_end_slide]))
                          if debug: print(sequence[refinedpos_start_slide:refinedpos_end_slide][-1])
                          refindpos_end_2nd.append(refinedpos_end_slide + 1)
                          pass
                      else:
                          # print("No P find!")
                          refindpos_end_2nd.append(refinedpos_end_slide)
                    except:
                        refindpos_end_2nd.append(refinedpos_end_slide)
                elif pos_index == (refinedpos[0].__len__() - 1):
                    if sequence[refinedpos_start_slide:refinedpos_end_slide][-1] == "P":
                        if debug: print("Find P in the last motif...")
                        if debug: print(len(sequence[refinedpos_start_slide:refinedpos_end_slide]))
                        if debug: print(sequence[refinedpos_start_slide:refinedpos_end_slide][-1])
                        refindpos_end_2nd.append(refinedpos_end_slide + 1)

                    else:
                        if debug: print("No P find in the last motif...")
                        refindpos_end_2nd.append(refinedpos_end_slide)
                pos_index += 1



        except Exception as e:
            print("Oops! Something wrong hanpens with the following sequence!")
            print(f"--->{s.id}")
            traceback.print_exc()
        finally:
            if debug: print(refindpos_end_2nd)

        # print(match_motif)
        # print(refinedpos)

        # extract motifs scores according to the match positions
        motif_score = []
        
        for line in open(ps_out_file,'r'):
          motif_score.append(float(line.split("\t")[-2]))
        # print(motif_score)
        parsed_ppr=[]
        for pos_start,pos_end,score in zip(refinedpos[0], refindpos_end_2nd,motif_score):
            mypos=sequence_[pos_start:pos_end]
            mypos.id=f'{s.id}_{pos_start+1}_{pos_end}'
            mypos.annotations={
                "PS_SCORE"      :   score,
                "PPR_STARTS"    :   pos_start+1,
                "PPR_ENDS"      :   pos_end,
                "FIFTH_AA"      :   sequence_[pos_start+4],
                "LAST_AA"       :   sequence_[pos_end-1],
                "PPR_CODE"      :   sequence_[pos_start+4]+sequence_[pos_end-1],
                "RNA_BASE"      :   code2rna_multi[sequence_[pos_start+4]+sequence_[pos_end-1]] if sequence_[pos_start+4]+sequence_[pos_end-1] in code2rna_multi.keys() else '?',
                
                               }
            parsed_ppr.append(mypos)
        return parsed_ppr
    except Exception as e:
        print("Oops! Something wrong hanpens with the following sequence!")
        print(f"--->{s.id}")
        traceback.print_exc()



In [3]:
#@title Enter the amino acid sequence to predict ⬇️
import glob

# define sequence
jobname = "ZmPPR10" #@param {type:"string"}
sequence = 'MEATGRGLFPNKPTLPAGPRKRGPLLPAAPPPPSPSSLPLDSLLLHLTAPAPAPAPAPRRSHQTPTPPHSFLSPDAQVLVLAISSHPLPTLAAFLASRRDELLRADITSLLKALELSGHWEWALALLRWAGKEGAADASALEMVVRALGREGQHDAVCALLDETPLPPGSRLDVRAYTTVLHALSRAGRYERALELFAELRRQGVAPTLVTYNVVLDVYGRMGRSWPRIVALLDEMRAAGVEPDGFTASTVIAACCRDGLVDEAVAFFEDLKARGHAPCVVTYNALLQVFGKAGNYTEALRVLGEMEQNGCQPDAVTYNELAGTYARAGFFEEAARCLDTMASKGLLPNAFTYNTVMTAYGNVGKVDEALALFDQMKKTGFVPNVNTYNLVLGMLGKKSRFTVMLEMLGEMSRSGCTPNRVTWNTMLAVCGKRGMEDYVTRVLEGMRSCGVELSRDTYNTLIAAYGRCGSRTNAFKMYNEMTSAGFTPCITTYNALLNVLSRQGDWSTAQSIVSKMRTKGFKPNEQSYSLLLQCYAKGGNVAGIAAIENEVYGSGAVFPSWVILRTLVIANFKCRRLDGMETAFQEVKARGYNPDLVIFNSMLSIYAKNGMYSKATEVFDSIKRSGLSPDLITYNSLMDMYAKCSESWEAEKILNQLKCSQTMKPDVVSYNTVINGFCKQGLVKEAQRVLSEMVADGMAPCAVTYHTLVGGYSSLEMFSEAREVIGYMVQHGLKPMELTYRRVVESYCRAKRFEEARGFLSEVSETDLDFDKKALEAYIEDAQFGR' #@param {type:"string"}
sequence=sequence.upper()
#@markdown ---

#@markdown OR upload your sequences in FASTA format

#@markdown Click the icon like a folder in left panel, upload your file into a subfolder named **jobs**.

#@markdown After uploading is over you must click the _Play Button_ ▶️ on each cell to continue the process.

#@markdown **this will ignore any input in boxes above.**

upload_my_fasta = True #@param {type:"boolean"}

generate_excel_report=True #@param {type:"boolean"}
download_full_report=True #@param {type:"boolean"}
#@markdown - `upload_my_fasta` - analysis uploaded file
#@markdown - `generate_excel_report` - generate a human-friendly report file in xlsx format
#@markdown - `download_full_report` - once the processing is finished, the full result will be zipped and downloaded automatically.
#@markdown ---
debug = False #@param {type:"boolean"}
#@markdown - `debug` - Some debug messages

#@markdown ---
#@markdown Experimental (manually correction to scan results)

fix_gap = False #@param {type:"boolean"}
#@markdown - `fix_gap` - fix gap in sequence scanning results. Turn it off so that the results will not be weird.

if not upload_my_fasta:
  open(f"./{JOBS_DIR}/{jobname}.fasta",'w').write(f'>{jobname}\n{sequence}')
else:
  if len(glob.glob(f'{JOBS_DIR}/*.fasta'))==0:
    raise FileNotFoundError("NO FASTA file is found!")


In [ ]:
#@title Process the FASTAs
fastas=glob.glob(f'{JOBS_DIR}/*.fasta')
if debug: print(fastas)

job_list=[s for f in fastas for s in SeqIO.parse(f,'fasta') ]
if debug: print(len(job_list))
total=len(job_list)

with tqdm.notebook.tqdm(total=total, bar_format=TQDM_BAR_FORMAT) as pbar:
  for f in fastas:
    for s in SeqIO.parse(f,'fasta'):
      ppr=ps_scan(f,s)
      pickle.dump(ppr,open(f'{RES_DIR_PICKLE}/{s.id}.pkl','wb'))
      pbar.update(1)

In [ ]:
#@title Generate report and download it! 
def generate_full_report():
  
  pkls_files=glob.glob(f'{RES_DIR_PICKLE}/*.pkl')
  process_id = time.strftime("%Y%m%d_%H%M%S", time.localtime())
  xlsx_filename = f"{RES_DIR_REPORT}/PPRCODE_RESULTS_{process_id}.xlsx" 
  test_book = xw.Workbook(xlsx_filename)

  # set workbook property
  test_book.set_properties({
                'title': 'PPRCODE PREDICTION',
                'subject': 'http://yinlab.hzau.edu.cn/pprcode/',
                'author': 'PPRCODE project',
                'manager': 'Yan et al.',
                'company': 'Yin Lab',
                'comments': 'Created with pprcode project.'})

  for pf in pkls_files:
    data = pickle.load(open(pf,'rb'))
    motif_starts  = [x.annotations[  "PPR_STARTS"   ] for x in data]
    motif_ends    = [x.annotations[  "PPR_ENDS"     ] for x in data]
    motif_sequence= [str(x.seq) for x in data]
    fifth_code    = [x.annotations[  "FIFTH_AA"     ] for x in data]
    last_code     = [x.annotations[  "LAST_AA"      ] for x in data]
    ppr_code      = [x.annotations[  "PPR_CODE"     ] for x in data]
    rna_base      = [x.annotations[  "RNA_BASE"     ] for x in data]
    motif_length  = [len(str(x.seq)) for x in data]
    motif_score   = [x.annotations[  "PS_SCORE"   ] for x in data]
    data_list=[[a,b,c,d,e,f,g,h,i] 
               for a,b,c,d,e,f,g,h,i in zip(
                   motif_starts,
                   motif_ends,
                   motif_sequence,
                   fifth_code,
                   last_code,
                   ppr_code,
                   rna_base,
                   motif_length,
                   motif_score)]
    seq_id=pathlib.Path(pf).resolve().stem
    try:
      write_excel_tables(test_book,seq_id,data_list)
    except:
      print("Oops! Something wrong hanpens when writing in to excel report!")
      print(seq_id)
  print("All jobs are accomplished.")
  test_book.close()

def write_excel_tables(test_book, sequence_name, ppr_total_list_reversed):
    # reversed ppr code result list for making xlsx file

    code_count_result_name = "PPR motifs and PPR codes: %s" % sequence_name
    format_title = test_book.add_format()
    format_content = test_book.add_format()
    format_title.set_font("Times New Roman")
    format_title.set_font_size(11)
    format_title.set_bold()
    format_title.set_bottom(2)

    format_content.set_font("Times New Roman")
    format_content.set_font_size(11)

    worksheet = test_book.add_worksheet(sequence_name)
    worksheet.set_column(0, 1, 10)
    worksheet.set_column(2, 2, 50)
    worksheet.set_column(3, 8, 10)

    worksheet.write(0, 0, code_count_result_name, format_title)
    for n in range(1, 9):
        worksheet.write(0, n, "", format_title)
    worksheet.write(1, 0, "Motif Start", format_title)
    worksheet.write(1, 1, "Motif End", format_title)
    worksheet.write(1, 2, "Motif Sequence", format_title)
    worksheet.write(1, 3, "Fifth amino acid", format_title)
    worksheet.write(1, 4, "Last amino acid", format_title)
    worksheet.write(1, 5, "PPR code", format_title)
    worksheet.write(1, 6, "RNA base", format_title)
    worksheet.write(1, 7, "Motif Length", format_title)
    worksheet.write(1, 8, "ProSite Score", format_title)

    row = 2
    col = 0

    rna = ""
    for ppr_list in ppr_total_list_reversed:
        # print(ppr_list)
        motif_start = ppr_list[0]
        motif_end = ppr_list[1]
        motif_sequence = ppr_list[2]
        fifth_code = ppr_list[3]
        last_code = ppr_list[4]
        ppr_code = ppr_list[5]
        rna_base = ppr_list[6]

        rna = ''.join([rna, " (", rna_base, ")"])

        motif_length = ppr_list[7]
        motif_score = ppr_list[8]
        worksheet.write(row, col, motif_start, format_content)
        worksheet.write(row, col + 1, motif_end, format_content)
        worksheet.write(row, col + 2, motif_sequence, format_content)
        worksheet.write(row, col + 3, fifth_code, format_content)
        worksheet.write(row, col + 4, last_code, format_content)
        worksheet.write(row, col + 5, ppr_code, format_content)
        worksheet.write(row, col + 6, rna_base, format_content)
        worksheet.write(row, col + 7, motif_length, format_content)
        worksheet.write(row, col + 8, motif_score, format_content)

        row += 1
        pass
    worksheet.write(row + 1, col, "The RNA predicted by this PPR sequence:", format_title)
    worksheet.write(row + 2, col, rna, format_title)
    worksheet.write(row + 4, col, "Notes:", format_title)
    worksheet.write(row + 5, col,
                    "'-' represents no correlated RNA bases for this PPR code identified by biochemical assays (EMSA, ITC);",
                    format_content)
    worksheet.write(row + 6, col,
                    "'?' represents unknown bases, as no biochemical assays have been performed to identify the correlated RNA bases for the PPR code.",
                    format_content)
    worksheet.write(row + 8, col, "If you find this tool useful to your research, please cite the following:",
                    format_title)
    worksheet.write(row + 9, col,
                    "Junjie Yan, Yinying Yao, Sixing Hong, Yan Yang, Cuicui Shen, Qunxia Zhang, Delin Zhang, Tingting Zou, Ping Yin; Delineation of pentatricopeptide repeat codes for target RNA prediction, Nucleic Acids Research, gkz075, https://doi.org/10.1093/nar/gkz075",
                    format_content)
    worksheet.write(row + 11, col, "Copyright: Yin lab", format_title)
    pass


if generate_excel_report: 
  generate_full_report()

zipped=f'PPRCODE_{time.strftime("%Y%m%d_%H%M%S", time.localtime())}.zip'
os.system(f"zip -FSr {zipped} {RES_DIR}")

if download_full_report:
  if IN_COLAB:
    files.download(f'{zipped}')
  else:
    print("this notebook appears to be running locally, to download click folder icon to the left, navigate to file, right click and download")

